In [4]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [5]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing lux.egg-info\PKG-INFO
writing dependency_links to lux.egg-info\dependency_links.txt
writing requirements to lux.egg-info\requires.txt
writing top-level names to lux.egg-info\top_level.txt
reading manifest file 'lux.egg-info\SOURCES.txt'
writing manifest file 'lux.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
copying sequentially\agent_personal_custom.py -> build\lib\sequentially
creating build\bdist.win-amd64\egg
creating build\bdist.win-amd64\egg\lux
creating build\bdist.win-amd64\egg\lux\env
copying build\lib\lux\env\agent.py -> build\bdist.win-amd64\egg\lux\env
copying build\lib\lux\env\lux_env.py -> build\bdist.win-amd64\egg\lux\env
creating build\bdist.win-amd64\egg\lux\env\rng
copying build\lib\lux\env\rng\rng.js -> build\bdist.win-amd64\egg\lux\env\rng
copying build\lib\lux\env\rng\rng.py -> build\bdist.win-amd64\egg\lux\env\rng
copying build\lib\lux\env\r

c:\Users\Duy-Khanh\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
c:\Users\Duy-Khanh\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\command\easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
zip_safe flag not set; analyzing archive contents...
lux.env.rng.__pycache__.rng.cpython-310: module references __file__
lux.game.__pycache__.game_constants.cpython-310: module references __file__
lux.tests.__pycache__.test_map.cpython-310: module references __file__
lux.tests.__pycache__.test_replay.cpython-310: module references __file__
sequentially.__pycache__.main.cpython-310: module references __file__


In [6]:
from stable_baselines3 import PPO  # pip install stable-baselines3
from stable_baselines3.common.callbacks import CheckpointCallback

from importlib import reload
import sequentially.agent_personal_custom
reload(sequentially.agent_personal_custom) # Reload the file from disk incase the above agent-writing cell block was edited
from sequentially.agent_personal_custom import AgentPersonalCustom

from lux.env.agent import Agent
from lux.env.lux_env import LuxEnvironment
from lux.game.constants import LuxMatchConfigs_Default


# Default Lux configs
configs = LuxMatchConfigs_Default

# Create a default opponent agent
opponent = Agent()

# Create a RL agent in training mode
player = AgentPersonalCustom(mode="train")

# Create the Lux environment
env = LuxEnvironment(configs=configs,
                     learning_agent=player,
                     opponent_agent=opponent)

# Define the model, you can pick other RL algos from Stable Baselines3 instead if you like
model = PPO("MlpPolicy",
                env,
                verbose=1,
                tensorboard_log="./lux_tensorboard/",
                learning_rate=0.001,
                gamma=0.999,
                gae_lambda=0.95,
                batch_size=2048 * 8,
                n_steps=2048 * 8
            )

# Define a learning rate schedule
# (number of steps, learning_rate)
schedule = [
    #(2000000, 0.01),
    (2, 0.001),
    (2, 0.0001),
]

Running in inference-only mode.
c:\Users\Duy-Khanh\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [7]:
from stable_baselines3.common.utils import get_schedule_fn
import time

print("Training model...")
start_time = time.time()
run_id = 1

# Save a checkpoint every 1M steps
checkpoint_callback = CheckpointCallback(save_freq=1000000,
                                         save_path='./sequentially/',
                                         name_prefix=f'rl_model_{run_id}')

# Train the policy
for steps, learning_rate in schedule:
    model.lr_schedule = get_schedule_fn(learning_rate)
    model.learn(total_timesteps=steps,
                callback=checkpoint_callback,
                reset_num_timesteps = False)
    
end_time = time.time()
elapsed_time = end_time - start_time

# Save final model
model.save(path=f'sequentially/model.zip')

print("Done training model.")
print(f"Total training time: {elapsed_time} seconds.")

Training model...
Logging to ./lux_tensorboard/PPO_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 96.3     |
|    ep_rew_mean     | 0.132    |
| time/              |          |
|    fps             | 909      |
|    iterations      | 1        |
|    time_elapsed    | 18       |
|    total_timesteps | 16384    |
---------------------------------
Logging to ./lux_tensorboard/PPO_0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 107      |
|    ep_rew_mean     | 0.104    |
| time/              |          |
|    fps             | 866      |
|    iterations      | 1        |
|    time_elapsed    | 18       |
|    total_timesteps | 32768    |
---------------------------------
Done training model.
Total training time: 37.79616928100586 seconds.


In [8]:
from kaggle_environments import make
import json
# make match with empty agent
env = make("lux_ai_2021", configuration={"seed": 5621242, "loglevel": 2, "annotations": True}, debug=False)

In [9]:
# Play the environment where the RL agent plays against itself
steps = env.run(["./sequentially/main.py", "./sequentially/main.py"])

In [10]:
# Render the match
env.render(mode="ipython", width=1200, height=800)